In [294]:
import nltk
from nltk.corpus import brown
from stat_parser import Parser, display_tree
import spacy


In [552]:
examples = [
    'Siri, please evaluate two plus three.',
    'Call my mom',
    'Can you play "Thriller" by Michael Jackson?',
    "Tell me about Elon Musk.",
    "I am doing the second assignment on PracticalAI"
]

parser = spacy.load("en")

In [313]:
pipes_structure = [SequencePipe([FindTokensPipe("VERB/nsubj/*"),
                                 NamedEntityFilterPipe(),
                                 NamedEntityExtractorPipe()]),
                   FindTokensPipe("VERB"),
                   AnyPipe([SequencePipe([FindTokensPipe("VBD/dobj/NNP"),
                                          AggregatePipe([NamedEntityFilterPipe("GPE"), 
                                                NamedEntityFilterPipe("PERSON")]),
                                          NamedEntityExtractorPipe()]),
                            SequencePipe([FindTokensPipe("VBD/**/*/pobj/NNP"),
                                          AggregatePipe([NamedEntityFilterPipe("LOC"), 
                                                NamedEntityFilterPipe("PERSON")]),
                                          NamedEntityExtractorPipe()])])]

In [488]:
from textpipeliner import PipelineEngine, Context
from textpipeliner.pipes import *

pipes_structure = [AggregatePipe([
                                  FindTokensPipe("VERB/nsubj/*")
                    ]),
                   FindTokensPipe("VERB"),
                   AnyPipe([
                           FindTokensPipe("VERB/dobj/[NOUN,PROPN,PRON]"),
                           AggregatePipe([
                               FindTokensPipe("VERB/dobj/**/**/*"),
                               FindTokensPipe("VERB/dobj/**")
                           ])
                    ]),
                   
                   FindTokensPipe("VERB/prep/*"),
                   SequencePipe([
                       AggregatePipe([
                            FindTokensPipe("VERB/prep/**/**/*"), 
                       ])
                   ])
                   
                  ]
                

engine = PipelineEngine(pipes_structure, Context(parser(doc)))
f = engine.process()

In [532]:
doc = parser(examples[1])

for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
          token.shape_, token.is_alpha, token.is_stop)

Call call VERB VB ROOT Xxxx True False
my -PRON- ADJ PRP$ poss xx True True
mom mom NOUN NN dobj xxx True False


In [533]:
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

Call VERB ROOT Call
my ADJ poss mom
mom NOUN dobj Call


In [553]:
def solve(sents):
    pipes_structure = [AggregatePipe([
                                  FindTokensPipe("VERB/nsubj/*")
                    ]),
                   FindTokensPipe("VERB"),
                   AnyPipe([
                           FindTokensPipe("VERB/dobj/[NOUN,PROPN,PRON]"),
                           AggregatePipe([
                               FindTokensPipe("VERB/dobj/**/**/*"),
                               FindTokensPipe("VERB/dobj/**")
                           ])
                    ]),
                   
                   FindTokensPipe("VERB/prep/*"),
                   SequencePipe([
                       AnyPipe([
                            FindTokensPipe("VERB/prep/**/**/*"), 
                            FindTokensPipe("VERB/dobj/[NOUN,PROPN,PRON]/**/ADJ")
                       ])
                   ])
                   
                  ]
                

    for sent in sents:
        print("Processing sentence:'{}'".format(sent))
        process_result = PipelineEngine(pipes_structure, Context(parser(sent))).process()[0]
        result = ['null' if len(res_item) == 0 else ", ".join([y.text for y in res_item]) for res_item in process_result]
        print("  Subject: {} \n  Action: {} \n  Object: {} \n  Preposition: {} \n  Properties: {}".format(*result))
        print("------------\n")

In [554]:
solve(examples)

Processing sentence:'Siri, please evaluate two plus three.'
  Subject: null 
  Action: evaluate 
  Object: plus, three, two 
  Preposition: null 
  Properties: null
------------

Processing sentence:'Call my mom'
  Subject: null 
  Action: Call 
  Object: mom 
  Preposition: null 
  Properties: my
------------

Processing sentence:'Can you play "Thriller" by Michael Jackson?'
  Subject: you 
  Action: play 
  Object: Thriller 
  Preposition: by 
  Properties: Jackson, Michael
------------

Processing sentence:'Tell me about Elon Musk.'
  Subject: null 
  Action: Tell 
  Object: me 
  Preposition: about 
  Properties: Musk, Elon
------------

Processing sentence:'I am doing the second assignment on PracticalAI'
  Subject: I 
  Action: doing 
  Object: assignment 
  Preposition: null 
  Properties: second
------------



In [557]:
def solve2(sents):
    
    for sent in sents:
        doc = parser(sent)
        print("Sentence: '{}'".format(sent))
        for token in doc:
            print(token.text, token.pos_, token.dep_, token.head.text)
        print("--------")

In [558]:
solve2(examples)

Sentence: 'Siri, please evaluate two plus three.'
Siri PROPN npadvmod evaluate
, PUNCT punct evaluate
please INTJ intj evaluate
evaluate VERB ROOT evaluate
two NUM dobj evaluate
plus CCONJ cc two
three NUM conj two
. PUNCT punct evaluate
--------
Sentence: 'Call my mom'
Call VERB ROOT Call
my ADJ poss mom
mom NOUN dobj Call
--------
Sentence: 'Can you play "Thriller" by Michael Jackson?'
Can VERB aux play
you PRON nsubj play
play VERB ROOT play
" PUNCT punct Thriller
Thriller PROPN dobj play
" PUNCT punct Thriller
by ADP prep play
Michael PROPN compound Jackson
Jackson PROPN pobj by
? PUNCT punct play
--------
Sentence: 'Tell me about Elon Musk.'
Tell VERB ROOT Tell
me PRON dobj Tell
about ADP prep Tell
Elon PROPN compound Musk
Musk PROPN pobj about
. PUNCT punct Tell
--------
Sentence: 'I am doing the second assignment on PracticalAI'
I PRON nsubj doing
am VERB aux doing
doing VERB ROOT doing
the DET det assignment
second ADJ amod assignment
assignment NOUN dobj doing
on ADP prep assi

In [563]:
from stat_parser import Parser
parser = Parser()
trees = parser.parse(examples[4])
print(trees)

(SBAR+S
  (NP (PRP I))
  (VP
    (VB am)
    (VBG doing)
    (NP
      (NP (DT the) (JJ second) (NN assignment))
      (PP (IN on) (NP (NN PracticalAI))))))
